In [24]:
import pdfplumber
import pandas as pd
from pypdf import PdfReader
import io
from openai import OpenAI

from PyPDF2 import PdfReader, PdfWriter


# --- CONFIGURATION ---
pdf_path = "PNBOne_statement.pdf"
pdf_password = "1749001500018763"
target_column = "Remarks"  # Adjust based on your PDF table column name
model = "gpt-3.5-turbo"  # or "gpt-4"
max_pages = 40
client = OpenAI(api_key=openai_api_key)

# --- UNLOCK PDF ---
def unlock_pdf(pdf_path, password):
    reader = PdfReader(pdf_path)
    if reader.is_encrypted:
        reader.decrypt(password)

    # Extract text as binary for pdfplumber
    with open(pdf_path, "rb") as f:
        encrypted = f.read()
    return io.BytesIO(encrypted)

# --- EXTRACT TABLES ---
import re

def extract_transactions_from_text(pdf_stream):
    # Regex pattern assuming:
    # - Date: dd/mm/yyyy
    # - Amount: 123.45 (no comma)
    # - Type: DR or CR
    # - Balance: comma allowed (e.g., 59,814.31)
    # - Remarks: everything after
    pattern = re.compile(r"""
        (?P<date>\d{2}/\d{2}/\d{4})\s+                     # Date
        (?P<amount>\d+\.\d{2})\s+                          # Amount
        (?P<type>DR|CR)\s+                                 # Type
        (?P<balance>[\d,]+\.\d{2})\s+                      # Balance
        (?P<remarks>.*)                                    # Remarks (greedy)
    """, re.VERBOSE)

    data = []

    with pdfplumber.open(pdf_stream, password=pdf_password) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if not text:
                continue
            for line in text.split("\n"):
                match = pattern.search(line)
                if match:
                    data.append(match.groupdict())

    df = pd.DataFrame(data)
    if not df.empty:
        df["Date"] = pd.to_datetime(df["date"], dayfirst=True)
        df["Amount"] = pd.to_numeric(df["amount"])
        df["Balance"] = pd.to_numeric(df["balance"].str.replace(",", ""))
        df["Remarks"] = df["remarks"].str.strip()
        df["Type"] = df["type"]
        df = df[["Date", "Amount", "Type", "Balance", "Remarks"]]
    return df




# --- LLM SHORT REMARK ---
def get_short_remark(remark):
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "Summarize bank transaction remarks into a short one-line description and assign a category (like Food, Travel, Salary, Shopping, etc.)"},
                {"role": "user", "content": f"Remark: {remark}"}
            ],
            temperature=0.5
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"LLM Error: {str(e)}"

# --- MAIN PIPELINE ---
stream = unlock_pdf(pdf_path, pdf_password)
df = extract_transactions_from_text(stream)


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

In [26]:
df.head(100)

,Date,Amount,Type,Balance,Remarks
0,2025-05-23,310.00,DR,59814.31,UPI/105295215340/P2M/ka57f2225@cnrb/BMTC
1,2025-05-23,18210.00,DR,60124.31,UPI/100604883441/P2M/travel1paytm@hdfcbank/
2,2025-05-23,30421.91,DR,78334.31,UPI/514303301586/P2M/ccbillpay@ptybl/ICICI
3,2025-05-22,9000.00,DR,108756.22,UPI/105209271448/P2V/pankaj.singhal77-
4,2025-05-20,560.00,DR,117756.22,UPI/514002090408/P2M/amazonupi@apl/Amazon
...,...,...,...,...,...
95,2025-03-24,11674.00,DR,42706.23,UPI/100227831148/P2M/travel1paytm@hdfcbank/
96,2025-03-24,1341.90,DR,54380.23,UPI/100227779047/P2M/tata1mghealthcare.payu
97,2025-03-24,110.00,DR,55722.13,UPI/508349740938/P2V/9916195545@ibl/GULUR
98,2025-03-24,7000.00,DR,55832.13,UPI/508349158443/P2V/marilyn3112-


In [27]:

# Apply LLM to each row's remark column (adjust column name as needed)
df["short_remark"] = df["Remarks"].apply(get_short_remark)

# Save results
df.to_csv("parsed_bank_statement_with_summary.csv", index=False)
print("✅ Finished processing. Output saved to 'parsed_bank_statement_with_summary.csv'")

KeyboardInterrupt: 